In [3]:
import socket
import sys
import time
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
import pyqtgraph as pg

In [2]:
# this version plots bar graphs from float
class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)
        
        HOST = ''   # Symbolic name meaning all available interfaces
        PORT = 8888 # Arbitrary non-privileged port
        try:
            self.s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            print('Socket created')
        except socket.error as msg:
            print('Failed to create socket. Error  : ' + str(msg))
            sys.exit()
        # Bind socket to local host and port
        try:
            self.s.bind((HOST, PORT))
            print('Socket bind complete')
        except socket.error as msg:
            print('Bind failed. Error : ' + str(msg))
            sys.exit()
        
        
        self.demonum = 2
        
        if self.demonum == 1:
            self.vectorsize = 11
            self.labelstrs = ['8PSK','AM-DSB','AM-SSB','BPSK','CPFSK','GFSK','PAM4','QAM16','QAM64','QPSK','WBFM'] # demo 1
            self.filterdata = True
        elif self.demonum == 2:
            self.vectorsize = 10    # set length of vector streaming from gnuradio (11 = demo1, 10 = demo2)
            self.labelstrs = ['0','1','2','3','4','5','6','7','8','9'] # demo 2
            self.filterdata = False
    
        self.labelstrdict = dict(enumerate(self.labelstrs))

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.plot()
        self.mainbox.layout().addWidget(self.canvas)
        
        ay = self.canvas.getAxis('bottom')
        ay.setTicks([self.labelstrdict.items()])

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)
        
        # bar graph
        self.bargraph = pg.BarGraphItem(x=np.arange(self.vectorsize), height=np.ones(self.vectorsize), width=0.8, brush='r')
        self.canvas.addItem(self.bargraph)


        #### Set Data  #####################

        self.counter = 0
        self.prevres = np.zeros((1, self.vectorsize))
        self.filtlen = 100
        self.filtbuffer = np.zeros((self.filtlen, self.vectorsize))
        self.fps = 0.
        self.lastupdate = time.time()

        #### Start  #####################
        self._update()

    def _update(self):
        
        data, addr = self.s.recvfrom(4*self.vectorsize)
        b = bytearray(data)
        ead = np.frombuffer(b, dtype=np.float32)
    
        self.ydata = ead
        
        if self.filterdata == True:
            self.filtbuffer[self.counter % self.filtlen] = self.ydata
            self.prevres = np.mean(self.filtbuffer,axis=0)
        else:
            self.prevres = self.ydata
        
        self.bargraph.setOpts(height=self.prevres)

        now = time.time()
        dt = (now-self.lastupdate)
        if dt <= 0:
            dt = 0.000000000001
        fps2 = 1.0 / dt
        self.lastupdate = now
        self.fps = self.fps * 0.9 + fps2 * 0.1
        tx = 'Mean Frame Rate:  {fps:.3f} FPS'.format(fps=self.fps )
        self.label.setText(tx)
        QtCore.QTimer.singleShot(1, self._update)
        self.counter += 1

In [ ]:
app = QtGui.QApplication(sys.argv)
thisapp = App()
thisapp.show()
sys.exit(app.exec_())

Socket created
Socket bind complete


In [4]:
# this version plots line plot from complex stream
class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)
        
        HOST = ''   # Symbolic name meaning all available interfaces
        PORT = 8888 # Arbitrary non-privileged port
        # Datagram (udp) socket
        try:
            self.s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            print('Socket created')
        except socket.error as msg:
            print('Failed to create socket. Error  : ' + str(msg))
            sys.exit()
        # Bind socket to local host and port
        try:
            self.s.bind((HOST, PORT))
        except socket.error as msg:
            print('Bind failed. Error : ' + str(msg))
            sys.exit()
        print('Socket bind complete')

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        #self.view = self.canvas.addViewBox()
        #self.view.setAspectLocked(True)
        #self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        #self.img = pg.ImageItem(border='w')
        #self.view.addItem(self.img)

        #self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')


        #### Set Data  #####################

        #self.x = np.linspace(0,9, num=10)
        #self.X,self.Y = np.meshgrid(self.x,self.x)

        self.counter = 0
        self.fps = 0.
        self.lastupdate = time.time()

        #### Start  #####################
        self._update()

    def _update(self):
        
        data, addr = self.s.recvfrom(1472)
        b = bytearray(data)
        ead = np.frombuffer(b, dtype=np.complex64)
    
        #self.data = np.sin(self.X/3.+self.counter/9.)*np.cos(self.Y/3.+self.counter/9.)
        self.ydata = np.real(ead)

        #self.img.setImage(self.data)
        self.h2.setData(self.ydata)

        now = time.time()
        dt = (now-self.lastupdate)
        if dt <= 0:
            dt = 0.000000000001
        fps2 = 1.0 / dt
        self.lastupdate = now
        self.fps = self.fps * 0.9 + fps2 * 0.1
        tx = 'Mean Frame Rate:  {fps:.3f} FPS'.format(fps=self.fps )
        self.label.setText(tx)
        QtCore.QTimer.singleShot(1, self._update)
        self.counter += 1

In [ ]:
app = QtGui.QApplication(sys.argv)
thisapp = App()
thisapp.show()
sys.exit(app.exec_())

Socket created
Socket bind complete
